In [2]:
import numpy as np
import cv2
import os

########## KNN CODE ############
def distance(v1, v2):
	# Euclidean 
	return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
	dist = []
	for i in range(train.shape[0]):
		ix = train[i, :-1]
		iy = train[i, -1]
		d = distance(test, ix)
		dist.append([d, iy])
	dk = sorted(dist, key=lambda x: x[0])[:k]
	labels = np.array(dk)[:, -1]
	output = np.unique(labels, return_counts=True)
	index = np.argmax(output[1])
	return output[0][index]
################################

cap = cv2.VideoCapture(0)
cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cascade_path)

dataset_path = "./face_dataset/"

face_data = []
labels = []
class_id = 0
names = {}

# Dataset preparation
for fx in os.listdir(dataset_path):
	if fx.endswith('.npy'):
		names[class_id] = fx[:-4]
		data_item = np.load(os.path.join(dataset_path, fx))
		face_data.append(data_item)
		target = class_id * np.ones((data_item.shape[0],))
		class_id += 1
		labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
print("Labels shape:", face_labels.shape)
print("Dataset shape:", face_dataset.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print("Trainset shape:", trainset.shape)

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
	ret, frame = cap.read()
	if ret == False:
		continue

	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)

	for face in faces:
		x, y, w, h = face
		offset = 5
		face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]

		if face_section is not None and face_section.size != 0:
			try:
				face_section = cv2.resize(face_section, (100, 100))
			except Exception as e:
				print(f"Resize failed: {e}")
				continue

			out = knn(trainset, face_section.flatten())
			cv2.putText(frame, names[int(out)], (x, y-10), font, 1, (255,0,0), 2, cv2.LINE_AA)
			cv2.rectangle(frame, (x, y), (x+w, y+h), (255,255,255), 2)
		else:
			print("Empty face_section, skipping frame.")
			continue

	cv2.imshow("Faces", frame)

	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()

Labels shape: (209, 1)
Dataset shape: (209, 30000)
Trainset shape: (209, 30001)
